In [2]:
from bs4 import BeautifulSoup
import requests as requests
import time
import re
from datetime import datetime

In [3]:
## Key variables valid for all the chuncks
timestamp_pattern = re.compile(r'(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.\d{3}Z)')
original_format = "%Y-%m-%dT%H:%M:%S.%fZ"
desired_format = "%Y-%m-%d %H:%M:%S"


## Test chunck
NB: need to solve the problem of some items not having a title (these are the comments). At the current time, I solved the problem with a `try - except`.

In [4]:
titles = []

start = time.time()

for i in range(1,11):
    
    try:
        url_posts = 'https://stacker.news/items/%d' % (i)
        response = requests.get(url_posts)
        soup = BeautifulSoup(response.text, 'html.parser')
    
        title = soup.find_all('a', class_='item_title__FH7AS text-reset me-2')
        
        for e in title:
            titles.append(e.get_text())
        
    except:
        continue

end = time.time()

print("The time of execution of above program is :",
      (end-start), "s")

## Collect post header with all the related data

In [47]:
url_posts = 'https://stacker.news/items/1'
response = requests.get(url_posts)
soup = BeautifulSoup(response.text, 'html.parser')

titolo = soup.find('a', class_='item_title__FH7AS text-reset me-2').get_text()
banner = soup.find('div', class_='item_other__MjgP3')

#post_stack = banner.find('span').text

banner_data = []

for i in banner.find_all('span'):
    banner_data.append(i.text)
    
#n_comments = soup.find('a', class_='text-reset position-relative').get_text()
#nym_creator = banner.find('a', ) 

print([titolo, banner_data])


['El Salvador Makes Bitcoin Legal Tender', ['603 sats', ' \\ ', '4 boost', ' \\ ', ' \\ ', '@k00b  11 Jun 2021', ' ', '', ' ', 'bitcoin', '']]


## Collect timestamp of post creation
The following code scans the parsed html response looking for an `<a>` tag that has an attribute `title` matching the timestamp format in the website.
Then it extracts it and turns it into a datetime object. 

In [20]:
url_posts = 'https://stacker.news/items/1'
response = requests.get(url_posts)
soup = BeautifulSoup(response.text, 'html.parser')

timestamp_element = soup.find('a', attrs={'title': timestamp_pattern})
timestamp_match = timestamp_pattern.search(timestamp_element['title']).group(1)

timestamp_datetime = datetime.strptime(timestamp_match, original_format)

print(timestamp_datetime)


2021-06-11 19:26:02.662000


## Scraping user profiles
Users profiles are scraped starting from the list of users extracted by scraping all the items (posts+comments)
The link to get the user profile is `https://stacker.news/$username$` 

In [28]:
## Test user - the creator of stacker.news

url_posts = 'https://stacker.news/k00b'
response = requests.get(url_posts)
soup = BeautifulSoup(response.text, 'html.parser')

# Nym 
nym = soup.find('div', class_='user-header_username__bqOV1').get_text()

# This could became a function
nym_tot_stacked = soup.find('div', class_='mb-2 ms-0 ms-sm-1 user-header_username__bqOV1 text-success').get_text()
regex_sats = r'[^\d]+'
nym_tot_stacked = re.sub(regex_sats, '', nym_tot_stacked)

# Nym address
nym_lnaddress = soup.find('button', class_='fw-bold ms-0 btn btn-primary', type='button').get_text()

# Stacking since -> first user appearance (measured by the first item he/she created)
# We need to decide the datatype for this index -> string is better
nym_first_item = soup.find('a', class_='ms-1').get_text()[1]

# Longest cowboy hat streak (README.md for cowboy-hat meaning)
nym_ch_streak = soup.find('small', class_='text-muted d-flex-inline').get_text()
#regex_ch = r'[^\d]+'
#nym_ch_streak = re.sub(regex_ch, '', nym_ch_streak)

print(nym_ch_streak)

stacking since: #1
